 ## Advancing programming : scrap_books 

In [4]:
! pip install Beautifulsoup4

In [1]:
import requests
url = "https://books.toscrape.com/"
response = requests.get(url)
html_source = response.text

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_source, 'html.parser')

In [3]:
title = soup.find('h1')
print(title.text)

All products


In [4]:
all_paragraphs = soup.find_all('p')
for p in all_paragraphs:
    print(p.text)








Â£51.77


    
        In stock
    








Â£53.74


    
        In stock
    








Â£50.10


    
        In stock
    








Â£47.82


    
        In stock
    








Â£54.23


    
        In stock
    








Â£22.65


    
        In stock
    








Â£33.34


    
        In stock
    








Â£17.93


    
        In stock
    








Â£22.60


    
        In stock
    








Â£52.15


    
        In stock
    








Â£13.99


    
        In stock
    








Â£20.66


    
        In stock
    








Â£17.46


    
        In stock
    








Â£52.29


    
        In stock
    








Â£35.02


    
        In stock
    








Â£57.25


    
        In stock
    








Â£23.88


    
        In stock
    








Â£37.59


    
        In stock
    








Â£51.33


    
        In stock
    








Â£45.17


    
        In stock
    



In [5]:
anchors = soup.select('div.image_container a')
for anchor in anchors:
    print(anchor['href'])

catalogue/a-light-in-the-attic_1000/index.html
catalogue/tipping-the-velvet_999/index.html
catalogue/soumission_998/index.html
catalogue/sharp-objects_997/index.html
catalogue/sapiens-a-brief-history-of-humankind_996/index.html
catalogue/the-requiem-red_995/index.html
catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html
catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html
catalogue/the-black-maria_991/index.html
catalogue/starving-hearts-triangular-trade-trilogy-1_990/index.html
catalogue/shakespeares-sonnets_989/index.html
catalogue/set-me-free_988/index.html
catalogue/scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html
catalogue/rip-it-up-and-start-again_986/index.html
catalogue/our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html

In [6]:
# Lists to store the data
titles = []
ratings = []
prices = []
availabilities = []
genres = []
descriptions = []

In [7]:
# Function to clean and convert the rating text
def convert_rating(rating):
    ratings_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    return ratings_dict.get(rating, 0)

# Loop through the pages
for page in range(1, 51):  # 50 pages * 20 books per page = 1000 books
    url = url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all books on the page
    books = soup.find_all('article', class_='product_pod')
    
    for book in books:
        # Title
        title = book.h3.a['title']
        titles.append(title)
        
        # Rating
        rating = book.p['class'][1]
        ratings.append(convert_rating(rating))
        
        # Price
   
        price = book.find('p', class_='price_color').text.replace('Â', '').strip('£')  # Remove the Â and £ symbols
        prices.append(float(price))

        
        # Availability
        availability = book.find('p', class_='instock availability').text.strip()
        availabilities.append(availability)
        
        # Link to book page to extract genre and description
        book_url = "https://books.toscrape.com/catalogue/" + book.h3.a['href']
        book_response = requests.get(book_url)
        book_soup = BeautifulSoup(book_response.text, 'html.parser')
        
        # Genre
        breadcrumb = book_soup.find('ul', class_='breadcrumb')
        if breadcrumb:
            genre = breadcrumb.find_all('a')[2].text
        else:
            genre = "No genre available"
        genres.append(genre)
        
        # Description
        try:
            description = book_soup.find('meta', attrs={'name': 'description'})['content'].strip()
        except TypeError:
            description = "No description available."
        descriptions.append(description)

In [9]:
import pandas as pd

In [10]:
# Create a DataFrame
books_df = pd.DataFrame({
    'Title': titles,
    'Rating': ratings,
    'Price (£)': prices,
    'Availability': availabilities,
    'Genre': genres,
    'Description': descriptions
})

In [11]:
books_df

,Title,Rating,Price (£),Availability,Genre,Description
0,A Light in the Attic,3,51.77,In stock,No genre available,No description available.
1,Tipping the Velvet,1,53.74,In stock,No genre available,No description available.
2,Soumission,1,50.10,In stock,No genre available,No description available.
3,Sharp Objects,4,47.82,In stock,No genre available,No description available.
4,Sapiens: A Brief History of Humankind,5,54.23,In stock,No genre available,No description available.
...,...,...,...,...,...,...
995,Our Band Could Be Your Life: Scenes from the A...,3,57.25,In stock,No genre available,No description available.
996,Olio,1,23.88,In stock,No genre available,No description available.
997,Mesaerion: The Best Science Fiction Stories 18...,1,37.59,In stock,No genre available,No description available.
998,Libertarianism for Beginners,2,51.33,In stock,No genre available,No description available.


In [12]:
# Save to Excel
books_df.to_excel('books_scraped.xlsx', index=False)

print("Scraping completed and data saved to 'books_scraped.xlsx'.")

Scraping completed and data saved to 'books_scraped.xlsx'.
